In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from __init__ import *
import numpy as np
from export.saver import read_pickle, save_pickle
from read.svg_reader import SVGReader
from helpers.helpers import filter_none
from placement2.attract import *
from visuals.plotter import plot_general
from copy import deepcopy
import networkx as nx
from itertools import pairwise, accumulate, product
from functools import reduce
from operator import add
from helpers.helpers import sort_and_group_objects
from domains.domain import Domain
from placement2.neighbors import *
from placement2.connectivity import *
from fixes.id_problems import report_problems
from new_solutions.selection import *
from helpers.layout import Layout
import plotly.io as pio


In [29]:
layout: Layout = read_pickle("overlap_1011")
layout

Layout(domains={'bedroom_1': Domain(bedroom_1, x=[0.00-3.22], y=[3.15-5.95]), 'kitchen': Domain(kitchen, x=[3.22-5.02], y=[0.00-2.24]), 'm_bath': Domain(m_bath, x=[4.82-7.34], y=[4.38-5.68]), 'm_bedroom': Domain(m_bedroom, x=[7.34-9.47], y=[3.15-5.89]), 'm_closet': Domain(m_closet, x=[8.38-9.38], y=[2.36-3.15]), 'bath': Domain(bath, x=[5.02-7.34], y=[3.15-4.38]), 'dining': Domain(dining, x=[5.02-8.38], y=[0.00-3.15]), 'transit_space': Domain(transit_space, x=[3.22-5.02], y=[2.24-4.02]), 'closet': Domain(closet, x=[8.38-9.38], y=[-0.05-2.36]), 'bedroom_2': Domain(bedroom_2, x=[0.00-3.22], y=[0.00-3.15]), 'laundry': Domain(laundry, x=[3.22-5.02], y=[4.02-5.77])}, graphs=DiGraphs(Gx=<networkx.classes.digraph.DiGraph object at 0x1115c97b0>, Gy=<networkx.classes.digraph.DiGraph object at 0x1115c9090>))

In [30]:
plot_general(layout.domains)

In [35]:
report_problems(layout)[0].action_details

[laundry-WEST-0.20, m_bath-EAST-0.20]

In [37]:
report_problems(layout)[0].action_details[1].action_types

[<ActionType.PUSH: 1>, <ActionType.SQUEEZE: 4>]

In [36]:
report_problems(layout)[0].action_details[1].direction

<Direction.EAST: 2>

In [6]:
case  = "amber_a_f01.svg"
sv = SVGReader(case)
sv.run()
# ag = AdjacencyGenerator(sv.layout, buffer_size=0.17)
# ag.run() # TODO shouldnt run if sv.layout is empty
# ag.draw_graph()
domains = sv.layout.domains
pos = create_pos(domains)


In [7]:
plot_general(sv.layout.domains)


In [8]:
ad, graphs = adjust_domains(domains)
Gx, Gy = graphs

## running fixer

In [9]:
plot_general(ad)

In [38]:
report_problems(Layout(ad, graphs))

[Problem(ix=0, HOLE, ['laundry', 'bath'], resolved=False,  n_actions=2,
 Problem(ix=1, HOLE, ['m_bath', 'm_bedroom'], resolved=False,  n_actions=2,
 Problem(ix=2, HOLE, ['m_closet', 'm_bedroom'], resolved=False,  n_actions=2]

In [39]:
playout = Layout(ad, graphs)

In [40]:
re = Reporter(playout)
re.run()

In [41]:
c = Cook(re)

initializing.. 0
running again -> 1
next best layout m_closet-STRETCH-for-HOLE near ['m_closet', 'm_bedroom']2-PS:1.18-S:1.59


In [42]:
c.run_again()

running again -> 2
next best layout m_bath-STRETCH-for-HOLE near ['m_bath', 'm_bedroom']1-PS:0.80-S:0.90


In [43]:
c.run_again()

running again -> 3
next best layout laundry-STRETCH-for-HOLE near ['laundry', 'bath']1-PS:0.80-S:0.90


In [44]:
c.run_again()

running again -> 4
next best layout m_bath-SQUEEZE-for-OVERLAP near ['laundry', 'm_bath']0-PS:0.40-S:0.20


In [45]:
c.run_again()

No more results!


In [46]:
c.run_again()

No more results!


In [47]:
c.plot_all()

n_rows 2, n_sols: 4
len sols = 4+1. Len indices = 6


In [48]:
c.bl_hist[-2]

node: laundry, action: STRETCH, summary: Counter({'OVERLAP': 1}), # unres probs: 1

In [49]:
plot_general(c.bl_hist[-2].layout.domains)

In [50]:
# save_pickle(c.bl_hist[-1].layout, "overlap_1011")